Name:Tushar  
Roll no:102303206  
Assignment2(UCS547)

Q1: Identify !, %, and %% used in cell in Google Colab.

In [1]:
!echo "Running with !"

%time sum(range(1000000))

Running with !
CPU times: user 20.9 ms, sys: 0 ns, total: 20.9 ms
Wall time: 21.4 ms


499999500000

In [2]:
%%bash
echo "Running with %%"
ls

Running with %%
sample_data


Q2: Identify all key nvidia-smi commands with multiple options

In [2]:
!nvidia-smi

!nvidia-smi --query-gpu=name,temperature.gpu,utilization.gpu,memory.total,memory.used --format=csv,noheader,nounits

!nvidia-smi --query-compute-apps=pid,used_memory --format=csv,noheader

Fri Feb 13 10:56:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Q3: Debug common CUDA errors (zero output, incorrect indexing, PTX errors)

In [3]:
%%writefile debug_cuda.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void add(int *a, int *b, int *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}

int main() {
    int n = 5;
    int h_a[5] = {1,2,3,4,5};
    int h_b[5] = {5,4,3,2,1};
    int h_c[5];

    int *d_a, *d_b, *d_c;

    cudaMalloc(&d_a, n*sizeof(int));
    cudaMalloc(&d_b, n*sizeof(int));
    cudaMalloc(&d_c, n*sizeof(int));

    cudaMemcpy(d_a, h_a, n*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n*sizeof(int), cudaMemcpyHostToDevice);

    add<<<1,5>>>(d_a, d_b, d_c, n);
    cudaDeviceSynchronize();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
        printf("CUDA Error: %s\n", cudaGetErrorString(err));

    cudaMemcpy(h_c, d_c, n*sizeof(int), cudaMemcpyDeviceToHost);

    for(int i=0;i<n;i++)
        printf("%d ", h_c[i]);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

Writing debug_cuda.cu


In [4]:
!nvcc debug_cuda.cu -o debug_cuda

nvcc warning : Support for offline compilation for architectures prior to '<compute/sm/lto>_75' will be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [5]:
!nvcc -arch=sm_75 debug_cuda.cu -o debug_cuda

In [6]:
!./debug_cuda

6 6 6 6 6 

Q4: Write a CUDA C/C++ program to demonstrate GPU kernel execu'on and thread indexing.  

a. Launch a CUDA kernel using: 1 block and 8 threads

In [7]:
%%writefile kernel_index.cu
#include <stdio.h>

__global__ void printThreadInfo() {
    int thread_id = threadIdx.x;
    int block_id = blockIdx.x;
    int global_id = blockIdx.x * blockDim.x + threadIdx.x;

    printf("Block ID: %d | Thread ID: %d | Global ID: %d\n",
            block_id, thread_id, global_id);
}

int main() {
    printThreadInfo<<<1, 8>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing kernel_index.cu


In [18]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets kernel_index.cu -o kernel_index

In [9]:
!./kernel_index

Block ID: 0 | Thread ID: 0 | Global ID: 0
Block ID: 0 | Thread ID: 1 | Global ID: 1
Block ID: 0 | Thread ID: 2 | Global ID: 2
Block ID: 0 | Thread ID: 3 | Global ID: 3
Block ID: 0 | Thread ID: 4 | Global ID: 4
Block ID: 0 | Thread ID: 5 | Global ID: 5
Block ID: 0 | Thread ID: 6 | Global ID: 6
Block ID: 0 | Thread ID: 7 | Global ID: 7


b. Each thread must print: Hello from GPU thread global_thread_id

In [10]:
%%writefile hello_gpu.cu
#include <stdio.h>

__global__ void helloKernel() {
    int global_id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from GPU thread %d\n", global_id);
}

int main() {
    helloKernel<<<1, 8>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing hello_gpu.cu


In [19]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets hello_gpu.cu -o hello_gpu

In [12]:
!./hello_gpu

Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7


c. Compute the global thread ID using: global_thread_id = blockIdx.x * blockDim.x +
threadIdx.x

In [13]:
%%writefile global_id.cu
#include <stdio.h>

__global__ void computeGlobalID() {
    int global_thread_id = blockIdx.x * blockDim.x + threadIdx.x;

    printf("Block: %d | Thread: %d | Global Thread ID: %d\n",
           blockIdx.x, threadIdx.x, global_thread_id);
}

int main() {
    computeGlobalID<<<1, 8>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing global_id.cu


In [16]:
!nvcc -arch=sm_75 -Wno-deprecated-gpu-targets global_id.cu -o global_id

In [17]:
!./global_id

Block: 0 | Thread: 0 | Global Thread ID: 0
Block: 0 | Thread: 1 | Global Thread ID: 1
Block: 0 | Thread: 2 | Global Thread ID: 2
Block: 0 | Thread: 3 | Global Thread ID: 3
Block: 0 | Thread: 4 | Global Thread ID: 4
Block: 0 | Thread: 5 | Global Thread ID: 5
Block: 0 | Thread: 6 | Global Thread ID: 6
Block: 0 | Thread: 7 | Global Thread ID: 7


d. Clearly separate: Host code (CPU) & Device code (GPU kernel)

In [20]:
%%writefile host_device_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

/* =========================
   Device Code (GPU Kernel)
   ========================= */
__global__ void helloKernel() {
    int global_thread_id = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Hello from GPU thread %d\n", global_thread_id);
}

/* =========================
   Host Code (CPU)
   ========================= */
int main() {

    printf("Launching GPU Kernel...\n");

    helloKernel<<<1, 8>>>();

    cudaDeviceSynchronize();

    printf("Kernel Execution Completed.\n");

    return 0;
}

Writing host_device_demo.cu


In [21]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets host_device_demo.cu -o host_device_demo

In [22]:
!./host_device_demo

Launching GPU Kernel...
Hello from GPU thread 0
Hello from GPU thread 1
Hello from GPU thread 2
Hello from GPU thread 3
Hello from GPU thread 4
Hello from GPU thread 5
Hello from GPU thread 6
Hello from GPU thread 7
Kernel Execution Completed.


Q5: Write a CUDA program to demonstrate host and device memory separation.  

a. Create an integer array of size 5 on the host (CPU).

In [23]:
%%writefile memory_separation.cu
#include <stdio.h>
#include <cuda_runtime.h>

/* =========================
   Device Code (GPU Kernel)
   ========================= */
__global__ void printDeviceArray(int *d_arr) {
    int i = threadIdx.x;
    printf("Device Array[%d] = %d\n", i, d_arr[i]);
}

/* =========================
   Host Code (CPU)
   ========================= */
int main() {

    // Host memory (CPU)
    int h_arr[5] = {10, 20, 30, 40, 50};

    // Device memory (GPU)
    int *d_arr;
    cudaMalloc((void**)&d_arr, 5 * sizeof(int));

    // Copy Host → Device
    cudaMemcpy(d_arr, h_arr, 5 * sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel (1 block, 5 threads)
    printDeviceArray<<<1, 5>>>(d_arr);
    cudaDeviceSynchronize();

    cudaFree(d_arr);

    return 0;
}

Writing memory_separation.cu


In [24]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets memory_separation.cu -o memory_separation

In [25]:
!./memory_separation

Device Array[0] = 10
Device Array[1] = 20
Device Array[2] = 30
Device Array[3] = 40
Device Array[4] = 50


b. Allocate corresponding memory on the device (GPU) using cudaMalloc().

In [31]:
%%writefile memory_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void modifyArray(int *d_arr) {
    int i = threadIdx.x;
    d_arr[i] = d_arr[i] * 2;
}

int main() {

    int h_arr[5] = {1, 2, 3, 4, 5};
    int *d_arr;

    printf("Host Array Before GPU:\n");
    for(int i = 0; i < 5; i++)
        printf("%d ", h_arr[i]);
    printf("\n");

    cudaMalloc((void**)&d_arr, 5 * sizeof(int));

    cudaMemcpy(d_arr, h_arr, 5 * sizeof(int), cudaMemcpyHostToDevice);

    modifyArray<<<1,5>>>(d_arr);
    cudaDeviceSynchronize();

    cudaMemcpy(h_arr, d_arr, 5 * sizeof(int), cudaMemcpyDeviceToHost);

    printf("Host Array After GPU Modification:\n");
    for(int i = 0; i < 5; i++)
        printf("%d ", h_arr[i]);
    printf("\n");

    cudaFree(d_arr);

    return 0;
}

Writing memory_demo.cu


In [34]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets memory_demo.cu -o memory_demo

In [35]:
!./memory_demo

Host Array Before GPU:
1 2 3 4 5 
Host Array After GPU Modification:
2 4 6 8 10 


c. Copy data from host to device using cudaMemcpy().

In [36]:
%%writefile memcpy_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

int main() {

    // Host memory (CPU)
    int h_arr[5] = {10, 20, 30, 40, 50};

    // Device memory pointer (GPU)
    int *d_arr;

    // Allocate memory on GPU
    cudaMalloc((void**)&d_arr, 5 * sizeof(int));

    // Copy data from Host to Device
    cudaMemcpy(d_arr, h_arr, 5 * sizeof(int), cudaMemcpyHostToDevice);

    printf("Data copied from Host to Device successfully.\n");

    cudaFree(d_arr);

    return 0;
}

Writing memcpy_demo.cu


In [37]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets memcpy_demo.cu -o memcpy_demo

In [38]:
!./memcpy_demo

Data copied from Host to Device successfully.


d. Launch a kernel where GPU threads print values from device memory.

In [39]:
%%writefile memcpy_demo.cu
#include <stdio.h>
#include <cuda_runtime.h>

int main() {

    // Host memory (CPU)
    int h_arr[5] = {10, 20, 30, 40, 50};

    // Device memory pointer (GPU)
    int *d_arr;

    // Allocate memory on GPU
    cudaMalloc((void**)&d_arr, 5 * sizeof(int));

    // Copy data from Host to Device
    cudaMemcpy(d_arr, h_arr, 5 * sizeof(int), cudaMemcpyHostToDevice);

    printf("Data copied from Host to Device successfully.\n");

    cudaFree(d_arr);

    return 0;
}

Overwriting memcpy_demo.cu


In [41]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets memcpy_demo.cu -o memcpy_demo

In [42]:
!./memcpy_demo

Data copied from Host to Device successfully.


e. Copy the data back from device to host and print it on CPU.

In [43]:
%%writefile host_device_full.cu
#include <stdio.h>
#include <cuda_runtime.h>

/* Device Code (GPU Kernel) */
__global__ void modifyArray(int *d_arr) {
    int i = threadIdx.x;
    d_arr[i] = d_arr[i] + 5;
}

/* Host Code (CPU) */
int main() {

    int h_arr[5] = {1, 2, 3, 4, 5};
    int *d_arr;

    printf("Host Array Before GPU:\n");
    for(int i = 0; i < 5; i++)
        printf("%d ", h_arr[i]);
    printf("\n");

    cudaMalloc((void**)&d_arr, 5 * sizeof(int));

    cudaMemcpy(d_arr, h_arr, 5 * sizeof(int), cudaMemcpyHostToDevice);

    modifyArray<<<1,5>>>(d_arr);
    cudaDeviceSynchronize();

    // Copy Device → Host
    cudaMemcpy(h_arr, d_arr, 5 * sizeof(int), cudaMemcpyDeviceToHost);

    printf("Host Array After Copying Back From GPU:\n");
    for(int i = 0; i < 5; i++)
        printf("%d ", h_arr[i]);
    printf("\n");

    cudaFree(d_arr);

    return 0;
}

Writing host_device_full.cu


In [44]:
!nvcc -arch=sm_75 -std=c++11 -Wno-deprecated-gpu-targets host_device_full.cu -o host_device_full

In [45]:
!./host_device_full

Host Array Before GPU:
1 2 3 4 5 
Host Array After Copying Back From GPU:
6 7 8 9 10 


Q6: Compare CPU times of List/tuple with Numpy arrays.

In [46]:
import time
import numpy as np

n = 10_000_000

# List
lst = list(range(n))
start = time.time()
lst = [x + 1 for x in lst]
end = time.time()
print("List time:", end - start)

# Tuple
tup = tuple(range(n))
start = time.time()
tup = tuple(x + 1 for x in tup)
end = time.time()
print("Tuple time:", end - start)

# NumPy Array
arr = np.arange(n)
start = time.time()
arr = arr + 1
end = time.time()
print("NumPy time:", end - start)

List time: 0.5285780429840088
Tuple time: 0.9325957298278809
NumPy time: 0.025868654251098633
